In [48]:
import pandas as pd
import altair as alt


df = pd.read_csv("/content/drive/MyDrive/csvs/lagrimasghibli.csv")

# Get all unique causes to define the domain of the color scale
unique_causes = sorted(pivot_long['Causa'].unique().tolist())

# Define a new set of more vibrant and warm colors, with red as most prominent
vibrant_palette = [
    '#FF0000', '#DC143C', '#B22222', '#FF4500', '#FFA500', # Strong reds and oranges
    '#FFD700', '#FFCC33', '#FF69B4', '#BB8FCE' # Clearer yellow, fuchsia, and a warmer purple for 'Sufrimiento ajeno'
]

# Create an initial mapping from cause to color
cause_to_color_map = {}
for i, cause in enumerate(unique_causes):
    cause_to_color_map[cause] = vibrant_palette[i % len(vibrant_palette)]

# Explicitly get the colors for 'Conflicto' and 'Dolor psicológico' and swap them
color_conflicto = cause_to_color_map['Conflicto']
color_dolor_psicologico = cause_to_color_map['Dolor psicológico']

cause_to_color_map['Conflicto'] = color_dolor_psicologico
cause_to_color_map['Dolor psicológico'] = color_conflicto

# Create the final domain and range lists for Altair
domain_list = list(cause_to_color_map.keys())
range_list = [cause_to_color_map[cause] for cause in domain_list]

stacked_bar_chart = (
    alt.Chart(pivot_long).mark_bar(opacity=0.7).encode( # Added opacity here
        x=alt.X('Ano:O', title='Año'),
        y=alt.Y('Duracion:Q', title='Duración total del llanto (segundos)'),
        color=alt.Color('Causa:N', title='Razón del llanto', scale=alt.Scale(domain=domain_list, range=range_list)), # Custom scale with domain and range
        order=alt.Order('Causa:N', sort='ascending'),
        tooltip=['Ano', 'Causa', 'Duracion']
    ).properties(
        width=1000, # Increased width for better visualization
        title='Evolución temporal de las causas de llanto y su duración en la obra de Miyazaki'
    ).configure_axis(
        labelColor='lightgray',
        titleColor='lightgray'
    ).configure_header(
        titleColor='lightgray',
        labelColor='lightgray'
    ).configure_legend(
        labelColor='lightgray',
        titleColor='lightgray'
    ).configure_title(
        color='lightgray'
    ).configure_view(
        stroke=None,
        fill='lightblue'
    )
)

stacked_bar_chart

alt.Chart(...)

In [28]:
import pandas as pd
import altair as alt

# -------------------------------------------------------
# 1) Cargar el CSV original
# -------------------------------------------------------
df = pd.read_csv("/content/drive/MyDrive/csvs/lagrimasghibli.csv")

# -------------------------------------------------------
# 2) Estandarizar nombres de columnas (opcional pero ordenado)
# -------------------------------------------------------
df.columns = (
    df.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.replace("á", "a")
    .str.replace("ó", "o")
    .str.replace("í", "i")
    .str.replace("é", "e")
    .str.replace("ú", "u")
    .str.replace("ñ", "n")
)

# -------------------------------------------------------
# 3) Asegurar que Ano sea numérico
# -------------------------------------------------------
df["Ano"] = pd.to_numeric(df["Ano"], errors="coerce")

# -------------------------------------------------------
# 4) Limpiar y estandarizar la columna 'Causa'
# -------------------------------------------------------
df["Causa"] = df["Causa"].str.strip() # Remove leading/trailing spaces
df["Causa"] = df["Causa"].replace({
    "Evento positivo ": "Evento positivo", # Unify 'Evento positivo ' with space
    "Dolor psicológico y dolor físico": "Dolor psicológico y físico", # Unify combined causes
    "Dolor psicológico/Dolor físico": "Dolor psicológico y físico"   # Unify combined causes
})

# -------------------------------------------------------
# 5) Crear tabla dinámica (pivot) para revisar datos (opcional)
# -------------------------------------------------------
pivot = df.pivot_table(
    values="Duracion_(seg.)",
    index="Ano",
    columns="Causa",
    aggfunc="sum",
    fill_value=0
)

# Convert pivot to long format for Altair chart
pivot_long = pivot.reset_index().melt(
    id_vars="Ano",
    var_name="Causa",
    value_name="Duracion"
)

print(pivot)  # solo para inspección

# -------------------------------------------------------
# 6) Gráfico de barras apiladas usando Altair
# -------------------------------------------------------

chart = (
    alt.Chart(pivot_long) # Use pivot_long for the chart
    .mark_bar()
    .encode(
        x=alt.X("Ano:O", title="Año"),
        y=alt.Y("Duracion:Q", title="Segundos totales llorando"), # Use 'Duracion' from pivot_long
        color=alt.Color("Causa:N", title="Razón del llanto"),
        tooltip=["Ano", "Causa", "Duracion"]
    )
    .properties(
        width=750,
        height=450,
        title="Razones del llanto en películas Ghibli a lo largo de los años"
    )
)

chart

Causa  Conflicto  Decepción personal  Dolor físico  Dolor psicológico  \
Ano                                                                     
1886           0                   0             0                  1   
1988           0                   0             0                 70   
1989           0                   0             0                  7   
1992           0                   0             3                  0   
1997           0                   0             0                  0   
2001           0                   0             0                 81   
2008          12                   0             0                 80   
2013           0                   0             0                 16   
2023           0                   3             0                 10   

Causa  Dolor psicológico y físico  Evento positivo  Pérdida  \
Ano                                                           
1886                            0                1        0   
1988   

alt.Chart(...)

In [27]:
print(df['Causa'].unique())

['Dolor psicológico' 'Evento positivo' 'Pérdida' 'Sufrimiento ajeno'
 'Dolor físico' 'Evento positivo ' 'Pérdida y dolor psicológico'
 'Dolor psicológico y dolor físico' 'Conflicto'
 'Dolor psicológico/Dolor físico' 'Decepción personal']
